In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM ,Dropout
from keras.callbacks import EarlyStopping
from keras import layers

from sklearn.metrics import mean_squared_error

rate=pd.read_csv('data/USD_KRW.csv')
rate.columns=['date', 'rate']
rate=rate.set_index('date')

rate_train_01=rate.loc['2018-02-01': '2019-01-31' ]
rate_train_02=rate.loc['2018-03-01': '2019-02-31' ]
rate_train_03=rate.loc['2018-04-01': '2019-03-31' ]
rate_train_04=rate.loc['2018-05-01': '2019-04-31' ]
rate_train_05=rate.loc['2018-06-01': '2019-05-31' ]
rate_train_06=rate.loc['2018-07-01': '2019-06-31' ]
rate_train_07=rate.loc['2018-08-01': '2019-07-31' ]
rate_train_08=rate.loc['2018-09-01': '2019-08-31' ]
rate_train_09=rate.loc['2018-10-01': '2019-09-31' ]
rate_train_10=rate.loc['2018-11-01': '2019-10-31' ]
rate_train_11=rate.loc['2018-12-01': '2019-11-31' ]
#rate_train_12=rate.loc['2019-01-01': '2019-12-31' ]

rate_test_01=rate.loc['2019-02-01':'2019-02-31']
rate_test_02=rate.loc['2019-03-01':'2019-03-31']
rate_test_03=rate.loc['2019-04-01':'2019-04-31']
rate_test_04=rate.loc['2019-05-01':'2019-05-31']
rate_test_05=rate.loc['2019-06-01':'2019-06-31']
rate_test_06=rate.loc['2019-07-01':'2019-07-31']
rate_test_07=rate.loc['2019-08-01':'2019-08-31']
rate_test_08=rate.loc['2019-09-01':'2019-09-31']
rate_test_09=rate.loc['2019-10-01':'2019-10-31']
rate_test_10=rate.loc['2019-11-01':'2019-11-31']
rate_test_11=rate.loc['2019-12-01':'2019-12-31']
#rate_test_12=rate.loc['2020-01-01':'2020-01-31']

kolib=pd.read_csv('data/KORIBOR.csv')
kolib=kolib[['date', '6month']]
kolib.columns=['date','krlibor']
kolib=kolib.set_index('date')

kolib_train_01=kolib.loc['2018-02-01': '2019-01-31' ]
kolib_train_02=kolib.loc['2018-03-01': '2019-02-31' ]
kolib_train_03=kolib.loc['2018-04-01': '2019-03-31' ]
kolib_train_04=kolib.loc['2018-05-01': '2019-04-31' ]
kolib_train_05=kolib.loc['2018-06-01': '2019-05-31' ]
kolib_train_06=kolib.loc['2018-07-01': '2019-06-31' ]
kolib_train_07=kolib.loc['2018-08-01': '2019-07-31' ]
kolib_train_08=kolib.loc['2018-09-01': '2019-08-31' ]
kolib_train_09=kolib.loc['2018-10-01': '2019-09-31' ]
kolib_train_10=kolib.loc['2018-11-01': '2019-10-31' ]
kolib_train_11=kolib.loc['2018-12-01': '2019-11-31' ]
#kolib_train_12=kolib.loc['2019-01-01': '2019-12-31' ]

kolib_test_01=kolib.loc['2019-02-01':'2019-02-31']
kolib_test_02=kolib.loc['2019-03-01':'2019-03-31']
kolib_test_03=kolib.loc['2019-04-01':'2019-04-31']
kolib_test_04=kolib.loc['2019-05-01':'2019-05-31']
kolib_test_05=kolib.loc['2019-06-01':'2019-06-31']
kolib_test_06=kolib.loc['2019-07-01':'2019-07-31']
kolib_test_07=kolib.loc['2019-08-01':'2019-08-31']
kolib_test_08=kolib.loc['2019-09-01':'2019-09-31']
kolib_test_09=kolib.loc['2019-10-01':'2019-10-31']
kolib_test_10=kolib.loc['2019-11-01':'2019-11-31']
kolib_test_11=kolib.loc['2019-12-01':'2019-12-31']
#kolib_test_12=kolib.loc['2020-01-01':'2020-01-31']

uslib=pd.read_csv('data/libor.csv')
uslib.columns=['date','uslibor']
uslib=uslib.set_index('date')

# 데이터에 일부분에 . 이 존재함. 
# 국가 공휴일은 삭제하고 필요한 값은 평균을 활용
del uslib['uslibor']['2018-12-25']
del uslib['uslibor']['2019-01-01']
del uslib['uslibor']['2019-05-27']
del uslib['uslibor']['2019-12-25']
del uslib['uslibor']['2020-01-01']

uslib['uslibor'].loc['2018-03-30'] = (float(uslib['uslibor'].loc['2018-03-29']) * 2/3 
                                            + float(uslib['uslibor'].loc['2018-04-03']) * 1/3)

uslib['uslibor'].loc['2018-04-02'] = (float(uslib['uslibor'].loc['2018-03-29']) * 1/3 
                                            + float(uslib['uslibor'].loc['2018-04-03']) * 2/3)

uslib['uslibor'].loc['2018-05-07'] = (float(uslib['uslibor'].loc['2018-05-04']) * 1/2 
                                            + float(uslib['uslibor'].loc['2018-05-08']) * 1/2)

uslib['uslibor'].loc['2018-05-28'] = (float(uslib['uslibor'].loc['2018-05-25']) * 1/2 
                                            + float(uslib['uslibor'].loc['2018-05-29']) * 1/2)

uslib['uslibor'].loc['2018-08-27'] = (float(uslib['uslibor'].loc['2018-08-24']) * 1/2 
                                            + float(uslib['uslibor'].loc['2018-08-28']) * 1/2)

uslib['uslibor'].loc['2018-12-26'] = (float(uslib['uslibor'].loc['2018-12-24']) * 2/3 
                                            + float(uslib['uslibor'].loc['2018-12-27']) * 1/3)

uslib['uslibor'].loc['2019-04-19'] = (float(uslib['uslibor'].loc['2019-04-23']) * 2/3 
                                            + float(uslib['uslibor'].loc['2019-04-18']) * 1/3)

uslib['uslibor'].loc['2019-04-22'] = (float(uslib['uslibor'].loc['2019-04-18']) * 2/3 
                                            + float(uslib['uslibor'].loc['2019-04-23']) * 1/3)

uslib['uslibor'].loc['2019-05-06'] = (float(uslib['uslibor'].loc['2019-05-03']) * 1/2
                                            + float(uslib['uslibor'].loc['2019-05-07']) * 1/2)

uslib['uslibor'].loc['2019-08-26'] = (float(uslib['uslibor'].loc['2019-08-23']) * 1/2
                                            + float(uslib['uslibor'].loc['2019-08-27']) * 1/2)

uslib['uslibor'].loc['2019-12-26'] = (float(uslib['uslibor'].loc['2019-12-24']) * 2/3 
                                            + float(uslib['uslibor'].loc['2019-12-27']) * 1/3)

# 값이 문자형으로 되어있어서 실수형으로 변경
uslib['uslibor']=uslib['uslibor'].apply(pd.to_numeric)

uslib_train_01=uslib.loc['2018-02-01': '2019-01-31' ]
uslib_train_02=uslib.loc['2018-03-01': '2019-02-31' ]
uslib_train_03=uslib.loc['2018-04-01': '2019-03-31' ]
uslib_train_04=uslib.loc['2018-05-01': '2019-04-31' ]
uslib_train_05=uslib.loc['2018-06-01': '2019-05-31' ]
uslib_train_06=uslib.loc['2018-07-01': '2019-06-31' ]
uslib_train_07=uslib.loc['2018-08-01': '2019-07-31' ]
uslib_train_08=uslib.loc['2018-09-01': '2019-08-31' ]
uslib_train_09=uslib.loc['2018-10-01': '2019-09-31' ]
uslib_train_10=uslib.loc['2018-11-01': '2019-10-31' ]
uslib_train_11=uslib.loc['2018-12-01': '2019-11-31' ]
#uslib_train_12=uslib.loc['2019-01-01': '2019-12-31' ]

uslib_test_01=uslib.loc['2019-02-01':'2019-02-31']
uslib_test_02=uslib.loc['2019-03-01':'2019-03-31']
uslib_test_03=uslib.loc['2019-04-01':'2019-04-31']
uslib_test_04=uslib.loc['2019-05-01':'2019-05-31']
uslib_test_05=uslib.loc['2019-06-01':'2019-06-31']
uslib_test_06=uslib.loc['2019-07-01':'2019-07-31']
uslib_test_07=uslib.loc['2019-08-01':'2019-08-31']
uslib_test_08=uslib.loc['2019-09-01':'2019-09-31']
uslib_test_09=uslib.loc['2019-10-01':'2019-10-31']
uslib_test_10=uslib.loc['2019-11-01':'2019-11-31']
uslib_test_11=uslib.loc['2019-12-01':'2019-12-31']
#uslib_test_12=uslib.loc['2020-01-01':'2020-01-31']

snp500=pd.read_csv('data/S&P500.csv')
snp500 = snp500[::-1]
snp500=snp500[['날짜','종가']]
snp500.columns=['date','snp500']


snp500_date=[]
for i in snp500['date']:
    snp500_date.append(i.replace('년 ','-').replace('월 ','-').replace('일',''))

snp500_price=[]
for i in snp500['snp500']:
    i=i.replace(',','')
    snp500_price.append(np.float(i))

snp500['date']=snp500_date
snp500['snp500']=snp500_price
snp500=snp500.set_index('date')

snp500_train_01=snp500.loc['2018-02-01': '2019-01-31' ]
snp500_train_02=snp500.loc['2018-03-01': '2019-02-31' ]
snp500_train_03=snp500.loc['2018-04-01': '2019-03-31' ]
snp500_train_04=snp500.loc['2018-05-01': '2019-04-31' ]
snp500_train_05=snp500.loc['2018-06-01': '2019-05-31' ]
snp500_train_06=snp500.loc['2018-07-01': '2019-06-31' ]
snp500_train_07=snp500.loc['2018-08-01': '2019-07-31' ]
snp500_train_08=snp500.loc['2018-09-01': '2019-08-31' ]
snp500_train_09=snp500.loc['2018-10-01': '2019-09-31' ]
snp500_train_10=snp500.loc['2018-11-01': '2019-10-31' ]
snp500_train_11=snp500.loc['2018-12-01': '2019-11-31' ]
#snp500_train_12=snp500.loc['2019-01-01': '2019-12-31' ]

snp500_test_01=snp500.loc['2019-02-01':'2019-02-31']
snp500_test_02=snp500.loc['2019-03-01':'2019-03-31']
snp500_test_03=snp500.loc['2019-04-01':'2019-04-31']
snp500_test_04=snp500.loc['2019-05-01':'2019-05-31']
snp500_test_05=snp500.loc['2019-06-01':'2019-06-31']
snp500_test_06=snp500.loc['2019-07-01':'2019-07-31']
snp500_test_07=snp500.loc['2019-08-01':'2019-08-31']
snp500_test_08=snp500.loc['2019-09-01':'2019-09-31']
snp500_test_09=snp500.loc['2019-10-01':'2019-10-31']
snp500_test_10=snp500.loc['2019-11-01':'2019-11-31']
snp500_test_11=snp500.loc['2019-12-01':'2019-12-31']
#snp500_test_12=snp500.loc['2020-01-01':'2020-01-31']

kospi200=pd.read_csv('data/KOSPI200.csv')
kospi200=kospi200[['날짜','종가']]
kospi200.columns=['date','kospi200']
kospi200 = kospi200[::-1]

kospi_date=[]
for i in kospi200['date']:
    kospi_date.append(i.replace('년 ','-').replace('월 ','-').replace('일',''))

kospi200['date']=kospi_date
kospi200=kospi200.set_index('date')

kospi200_train_01=kospi200.loc['2018-02-01': '2019-01-31' ]
kospi200_train_02=kospi200.loc['2018-03-01': '2019-02-31' ]
kospi200_train_03=kospi200.loc['2018-04-01': '2019-03-31' ]
kospi200_train_04=kospi200.loc['2018-05-01': '2019-04-31' ]
kospi200_train_05=kospi200.loc['2018-06-01': '2019-05-31' ]
kospi200_train_06=kospi200.loc['2018-07-01': '2019-06-31' ]
kospi200_train_07=kospi200.loc['2018-08-01': '2019-07-31' ]
kospi200_train_08=kospi200.loc['2018-09-01': '2019-08-31' ]
kospi200_train_09=kospi200.loc['2018-10-01': '2019-09-31' ]
kospi200_train_10=kospi200.loc['2018-11-01': '2019-10-31' ]
kospi200_train_11=kospi200.loc['2018-12-01': '2019-11-31' ]
#kospi200_train_12=kospi200.loc['2019-01-01': '2019-12-31' ]

kospi200_test_01=kospi200.loc['2019-02-01':'2019-02-31']
kospi200_test_02=kospi200.loc['2019-03-01':'2019-03-31']
kospi200_test_03=kospi200.loc['2019-04-01':'2019-04-31']
kospi200_test_04=kospi200.loc['2019-05-01':'2019-05-31']
kospi200_test_05=kospi200.loc['2019-06-01':'2019-06-31']
kospi200_test_06=kospi200.loc['2019-07-01':'2019-07-31']
kospi200_test_07=kospi200.loc['2019-08-01':'2019-08-31']
kospi200_test_08=kospi200.loc['2019-09-01':'2019-09-31']
kospi200_test_09=kospi200.loc['2019-10-01':'2019-10-31']
kospi200_test_10=kospi200.loc['2019-11-01':'2019-11-31']
kospi200_test_11=kospi200.loc['2019-12-01':'2019-12-31']
#kospi200_test_12=kospi200.loc['2020-01-01':'2020-01-31']

data_train_01=pd.concat([rate_train_01,kolib_train_01,uslib_train_01,snp500_train_01,kospi200_train_01], 
                       axis=1,  join = 'inner', sort=False)
data_test_01=pd.concat([rate_test_01,kolib_test_01,uslib_test_01,snp500_test_01,kospi200_test_01], 
                      axis=1,  join = 'inner', sort=False)
data_train_02=pd.concat([rate_train_02,kolib_train_02,uslib_train_02,snp500_train_02,kospi200_train_02], 
                       axis=1,  join = 'inner', sort=False)
data_test_02=pd.concat([rate_test_02,kolib_test_02,uslib_test_02,snp500_test_02,kospi200_test_02], 
                      axis=1,  join = 'inner', sort=False)
data_train_03=pd.concat([rate_train_03,kolib_train_03,uslib_train_03,snp500_train_03,kospi200_train_03], 
                       axis=1,  join = 'inner', sort=False)
data_test_03=pd.concat([rate_test_03,kolib_test_03,uslib_test_03,snp500_test_03,kospi200_test_03], 
                      axis=1,  join = 'inner', sort=False)
data_train_04=pd.concat([rate_train_04,kolib_train_04,uslib_train_04,snp500_train_04,kospi200_train_04], 
                       axis=1,  join = 'inner', sort=False)
data_test_04=pd.concat([rate_test_04,kolib_test_04,uslib_test_04,snp500_test_04,kospi200_test_04], 
                      axis=1,  join = 'inner', sort=False)
data_train_05=pd.concat([rate_train_05,kolib_train_05,uslib_train_05,snp500_train_05,kospi200_train_05], 
                       axis=1,  join = 'inner', sort=False)
data_test_05=pd.concat([rate_test_05,kolib_test_05,uslib_test_05,snp500_test_05,kospi200_test_05], 
                      axis=1,  join = 'inner', sort=False)
data_train_06=pd.concat([rate_train_06,kolib_train_06,uslib_train_06,snp500_train_06,kospi200_train_06], 
                       axis=1,  join = 'inner', sort=False)
data_test_06=pd.concat([rate_test_06,kolib_test_06,uslib_test_06,snp500_test_06,kospi200_test_06], 
                      axis=1,  join = 'inner', sort=False)
data_train_07=pd.concat([rate_train_07,kolib_train_07,uslib_train_07,snp500_train_07,kospi200_train_07], 
                       axis=1,  join = 'inner', sort=False)
data_test_07=pd.concat([rate_test_07,kolib_test_07,uslib_test_07,snp500_test_07,kospi200_test_07], 
                      axis=1,  join = 'inner', sort=False)
data_train_08=pd.concat([rate_train_08,kolib_train_08,uslib_train_08,snp500_train_08,kospi200_train_08], 
                       axis=1,  join = 'inner', sort=False)
data_test_08=pd.concat([rate_test_08,kolib_test_08,uslib_test_08,snp500_test_08,kospi200_test_08], 
                      axis=1,  join = 'inner', sort=False)
data_train_09=pd.concat([rate_train_09,kolib_train_09,uslib_train_09,snp500_train_09,kospi200_train_09], 
                       axis=1,  join = 'inner', sort=False)
data_test_09=pd.concat([rate_test_09,kolib_test_09,uslib_test_09,snp500_test_09,kospi200_test_09], 
                      axis=1,  join = 'inner', sort=False)
data_train_10=pd.concat([rate_train_10,kolib_train_10,uslib_train_10,snp500_train_10,kospi200_train_10], 
                       axis=1,  join = 'inner', sort=False)
data_test_10=pd.concat([rate_test_10,kolib_test_10,uslib_test_10,snp500_test_10,kospi200_test_10], 
                      axis=1,  join = 'inner', sort=False)
data_train_11=pd.concat([rate_train_11,kolib_train_11,uslib_train_11,snp500_train_11,kospi200_train_11], 
                       axis=1,  join = 'inner', sort=False)
data_test_11=pd.concat([rate_test_11,kolib_test_11,uslib_test_11,snp500_test_11,kospi200_test_11], 
                      axis=1,  join = 'inner', sort=False)
#ata_train_12=pd.concat([rate_train_12,kolib_train_12,uslib_train_12,snp500_train_12,kospi200_train_12], 
#                      axis=1,  join = 'inner', sort=False)
#ata_test_12=pd.concat([rate_test_12,kolib_test_12,uslib_test_12,snp500_test_12,kospi200_test_12], 
#                     axis=1,  join = 'inner', sort=False)
data_train_01['y']=data_train_01['rate'].shift(-1)
data_train_02['y']=data_train_02['rate'].shift(-1)
data_train_03['y']=data_train_03['rate'].shift(-1)
data_train_04['y']=data_train_04['rate'].shift(-1)
data_train_05['y']=data_train_05['rate'].shift(-1)
data_train_06['y']=data_train_06['rate'].shift(-1)
data_train_07['y']=data_train_07['rate'].shift(-1)
data_train_08['y']=data_train_08['rate'].shift(-1)
data_train_09['y']=data_train_09['rate'].shift(-1)
data_train_10['y']=data_train_10['rate'].shift(-1)
data_train_11['y']=data_train_11['rate'].shift(-1)
#data_train_12['y']=data_train_01['rate'].shift(-1)
data_test_01['y']=data_test_01['rate'].shift(-1)
data_test_02['y']=data_test_02['rate'].shift(-1)
data_test_03['y']=data_test_03['rate'].shift(-1)
data_test_04['y']=data_test_04['rate'].shift(-1)
data_test_05['y']=data_test_05['rate'].shift(-1)
data_test_06['y']=data_test_06['rate'].shift(-1)
data_test_07['y']=data_test_07['rate'].shift(-1)
data_test_08['y']=data_test_08['rate'].shift(-1)
data_test_09['y']=data_test_09['rate'].shift(-1)
data_test_10['y']=data_test_10['rate'].shift(-1)
data_test_11['y']=data_test_11['rate'].shift(-1)
#data_test_12['y']=data_test_12['rate'].shift(-1)
data_train_01['y'][-1]=rate_test_01.iloc[0]
data_train_01=data_train_01[['krlibor','uslibor','snp500','kospi200','y']]

data_train_02['y'][-1]=rate_test_02.iloc[0]
data_train_02=data_train_02[['krlibor','uslibor','snp500','kospi200','y']]

data_train_03['y'][-1]=rate_test_03.iloc[0]
data_train_03=data_train_03[['krlibor','uslibor','snp500','kospi200','y']]

data_train_04['y'][-1]=rate_test_04.iloc[0]
data_train_04=data_train_04[['krlibor','uslibor','snp500','kospi200','y']]

data_train_05['y'][-1]=rate_test_05.iloc[0]
data_train_05=data_train_05[['krlibor','uslibor','snp500','kospi200','y']]

data_train_06['y'][-1]=rate_test_06.iloc[0]
data_train_06=data_train_06[['krlibor','uslibor','snp500','kospi200','y']]

data_train_07['y'][-1]=rate_test_07.iloc[0]
data_train_07=data_train_07[['krlibor','uslibor','snp500','kospi200','y']]

data_train_08['y'][-1]=rate_test_08.iloc[0]
data_train_08=data_train_08[['krlibor','uslibor','snp500','kospi200','y']]

data_train_09['y'][-1]=rate_test_09.iloc[0]
data_train_09=data_train_09[['krlibor','uslibor','snp500','kospi200','y']]

data_train_10['y'][-1]=rate_test_10.iloc[0]
data_train_10=data_train_10[['krlibor','uslibor','snp500','kospi200','y']]

data_train_11['y'][-1]=rate_test_11.iloc[0]
data_train_11=data_train_11[['krlibor','uslibor','snp500','kospi200','y']]

#data_train_12['y'][-1]=rate_test_12.iloc[0]
#data_train_12=data_train_12[['krlibor','uslibor','snp500','kospi200','y']]

data_test_01['y'][-1]=rate_test_01.iloc[0]
data_test_01=data_test_01[['krlibor','uslibor','snp500','kospi200','y']]
data_test_01=data_test_01[:-1]

data_test_02['y'][-1]=rate_test_02.iloc[0]
data_test_02=data_test_02[['krlibor','uslibor','snp500','kospi200','y']]
data_test_02=data_test_02[:-1]

data_test_03['y'][-1]=rate_test_03.iloc[0]
data_test_03=data_test_03[['krlibor','uslibor','snp500','kospi200','y']]
data_test_03=data_test_03[:-1]

data_test_04['y'][-1]=rate_test_04.iloc[0]
data_test_04=data_test_04[['krlibor','uslibor','snp500','kospi200','y']]
data_test_04=data_test_04[:-1]

data_test_05['y'][-1]=rate_test_05.iloc[0]
data_test_05=data_test_05[['krlibor','uslibor','snp500','kospi200','y']]
data_test_05=data_test_05[:-1]

data_test_06['y'][-1]=rate_test_06.iloc[0]
data_test_06=data_test_06[['krlibor','uslibor','snp500','kospi200','y']]
data_test_06=data_test_06[:-1]

data_test_07['y'][-1]=rate_test_07.iloc[0]
data_test_07=data_test_07[['krlibor','uslibor','snp500','kospi200','y']]
data_test_07=data_test_07[:-1]

data_test_08['y'][-1]=rate_test_08.iloc[0]
data_test_08=data_test_08[['krlibor','uslibor','snp500','kospi200','y']]
data_test_08=data_test_08[:-1]

data_test_09['y'][-1]=rate_test_09.iloc[0]
data_test_09=data_test_09[['krlibor','uslibor','snp500','kospi200','y']]
data_test_09=data_test_09[:-1]

data_test_10['y'][-1]=rate_test_10.iloc[0]
data_test_10=data_test_10[['krlibor','uslibor','snp500','kospi200','y']]
data_test_10=data_test_10[:-1]

data_test_11['y'][-1]=rate_test_11.iloc[0]
data_test_11=data_test_11[['krlibor','uslibor','snp500','kospi200','y']]
data_test_11=data_test_11[:-1]

#data_test_12['y'][-1]=rate_test_12.iloc[0]
#data_test_12=data_test_12[['krlibor','uslibor','snp500','kospi200','y']]
#data_test_12=data_test_12[:-1]
train_input_01 = data_train_01[['krlibor','uslibor','snp500','kospi200']]
train_target_01 = data_train_01['y']
test_input_01 = data_test_01[['krlibor','uslibor','snp500','kospi200']]
test_target_01 = data_test_01['y']

train_input_02 = data_train_02[['krlibor','uslibor','snp500','kospi200']]
train_target_02 = data_train_02['y']
test_input_02 = data_test_02[['krlibor','uslibor','snp500','kospi200']]
test_target_02 = data_test_02['y']

train_input_03= data_train_03[['krlibor','uslibor','snp500','kospi200']]
train_target_03 = data_train_03['y']
test_input_03 = data_test_03[['krlibor','uslibor','snp500','kospi200']]
test_target_03 = data_test_03['y']

train_input_04= data_train_04[['krlibor','uslibor','snp500','kospi200']]
train_target_04 = data_train_04['y']
test_input_04 = data_test_04[['krlibor','uslibor','snp500','kospi200']]
test_target_04 = data_test_04['y']

train_input_05= data_train_05[['krlibor','uslibor','snp500','kospi200']]
train_target_05 = data_train_05['y']
test_input_05 = data_test_05[['krlibor','uslibor','snp500','kospi200']]
test_target_05 = data_test_05['y']

train_input_06= data_train_06[['krlibor','uslibor','snp500','kospi200']]
train_target_06 = data_train_06['y']
test_input_06 = data_test_06[['krlibor','uslibor','snp500','kospi200']]
test_target_06 = data_test_06['y']

train_input_07 = data_train_07[['krlibor','uslibor','snp500','kospi200']]
train_target_07 = data_train_07['y']
test_input_07 = data_test_07[['krlibor','uslibor','snp500','kospi200']]
test_target_07 = data_test_07['y']

train_input_08 = data_train_08[['krlibor','uslibor','snp500','kospi200']]
train_target_08 = data_train_08['y']
test_input_08 = data_test_08[['krlibor','uslibor','snp500','kospi200']]
test_target_08 = data_test_08['y']

train_input_09 = data_train_09[['krlibor','uslibor','snp500','kospi200']]
train_target_09 = data_train_09['y']
test_input_09 = data_test_09[['krlibor','uslibor','snp500','kospi200']]
test_target_09 = data_test_09['y']

train_input_10 = data_train_10[['krlibor','uslibor','snp500','kospi200']]
train_target_10 = data_train_10['y']
test_input_10 = data_test_10[['krlibor','uslibor','snp500','kospi200']]
test_target_10 = data_test_10['y']

train_input_11 = data_train_11[['krlibor','uslibor','snp500','kospi200']]
train_target_11 = data_train_11['y']
test_input_11 = data_test_11[['krlibor','uslibor','snp500','kospi200']]
test_target_11 = data_test_11['y']

#train_input_12 = data_train_12[['krlibor','uslibor','snp500','kospi200']]
#train_target_12 = data_train_12['y']
#test_input_12 = data_test_12[['krlibor','uslibor','snp500','kospi200']]
#test_target_12 = data_test_12['y']
mean_01 = train_input_01.mean(axis=0)
std_01 = train_input_01.std(axis=0)
train_input_01 -= mean_01
train_input_01 /= std_01
test_input_01 -= mean_01
test_input_01 /= std_01

mean_02 = train_input_02.mean(axis=0)
std_02 = train_input_02.std(axis=0)
train_input_02 -= mean_02
train_input_02 /= std_02
test_input_02 -= mean_02
test_input_02 /= std_02

mean_03 = train_input_03.mean(axis=0)
std_03 = train_input_03.std(axis=0)
train_input_03 -= mean_03
train_input_03 /= std_03
test_input_03 -= mean_03
test_input_03 /= std_03

mean_04 = train_input_04.mean(axis=0)
std_04 = train_input_04.std(axis=0)
train_input_04 -= mean_04
train_input_04 /= std_04
test_input_04 -= mean_04
test_input_04 /= std_04

mean_05 = train_input_05.mean(axis=0)
std_05 = train_input_05.std(axis=0)
train_input_05 -= mean_05
train_input_05 /= std_05
test_input_05 -= mean_05
test_input_05 /= std_05

mean_06 = train_input_06.mean(axis=0)
std_06 = train_input_06.std(axis=0)
train_input_06 -= mean_06
train_input_06 /= std_06
test_input_06 -= mean_06
test_input_06 /= std_06

mean_07 = train_input_07.mean(axis=0)
std_07 = train_input_07.std(axis=0)
train_input_07 -= mean_07
train_input_07 /= std_07
test_input_07 -= mean_07
test_input_07 /= std_07

mean_08 = train_input_08.mean(axis=0)
std_08 = train_input_08.std(axis=0)
train_input_08 -= mean_08
train_input_08 /= std_08
test_input_08 -= mean_08
test_input_08 /= std_08

mean_09 = train_input_09.mean(axis=0)
std_09 = train_input_09.std(axis=0)
train_input_09 -= mean_09
train_input_09 /= std_09
test_input_09 -= mean_09
test_input_09 /= std_09

mean_10 = train_input_10.mean(axis=0)
std_10 = train_input_10.std(axis=0)
train_input_10 -= mean_10
train_input_10 /= std_10
test_input_10 -= mean_10
test_input_10 /= std_10

mean_11 = train_input_11.mean(axis=0)
std_11 = train_input_11.std(axis=0)
train_input_11 -= mean_11
train_input_11 /= std_11
test_input_11 -= mean_11
test_input_11 /= std_11

#mean_12 = train_input_12.mean(axis=0)
#std_12 = train_input_12.std(axis=0)
#train_input_12 -= mean_12
#train_input_12 /= std_12
#test_input_12 -= mean_12
#test_input_12 /= std_12

train_input_01=np.array(train_input_01)
train_input_01=train_input_01.reshape(train_input_01.shape[0],4)

train_input_02=np.array(train_input_02)
train_input_02=train_input_02.reshape(train_input_02.shape[0],4)

train_input_03=np.array(train_input_03)
train_input_03=train_input_03.reshape(train_input_03.shape[0],4)

train_input_04=np.array(train_input_04)
train_input_04=train_input_04.reshape(train_input_04.shape[0],4)

train_input_05=np.array(train_input_05)
train_input_05=train_input_05.reshape(train_input_05.shape[0],4)

train_input_06=np.array(train_input_06)
train_input_06=train_input_06.reshape(train_input_06.shape[0],4)

train_input_07=np.array(train_input_07)
train_input_07=train_input_07.reshape(train_input_07.shape[0],4)

train_input_08=np.array(train_input_08)
train_input_08=train_input_08.reshape(train_input_08.shape[0],4)

train_input_09=np.array(train_input_09)
train_input_09=train_input_09.reshape(train_input_09.shape[0],4)

train_input_10=np.array(train_input_10)
train_input_10=train_input_10.reshape(train_input_10.shape[0],4)

train_input_11=np.array(train_input_11)
train_input_11=train_input_11.reshape(train_input_11.shape[0],4)

#train_input_12=np.array(train_input_12)
#train_input_12=train_input_12.reshape(train_input_12.shape[0],4)


Using TensorFlow backend.


-----------------

# 모델 작성하기 (DNN)

In [2]:
import tensorflow as tf

In [3]:
early_stopping = EarlyStopping(patience=50)

In [4]:
model_01 = Sequential()

model_01.add(layers.Dense(units=1024, input_shape=(4,), ))
model_01.add(Dropout(0.1))

model_01.add(layers.Dense(units=1024))
model_01.add(Dropout(0.1))


model_01.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_01.compile(optimizer='Adagrad', loss='mean_squared_error')
model_01.fit(train_input_01, train_target_01, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 165 samples, validate on 71 samples
Epoch 1/1000
165/165 [==============================] - 1s 3ms/step - loss: 1016088.9708 - val_loss: 3582023.3908
Epoch 2/1000
165/165 [==============================] - 0s 135us/step - loss: 272339.7689 - val_loss: 5033187.6655
Epoch 3/1000
165/165 [==============================] - 0s 151us/step - loss: 165213.0560 - val_loss: 3787740.1796
Epoch 4/1000
165/165 [==============================] - 0s 149us/step - loss: 135635.0338 - val_loss: 4001260.8415
Epoch 5/1000
165/165 [==============================] - 0s 146us/step - loss: 111218.8725 - val_loss: 3418946.7905
Epoch 6/1000
165/165 [==============================] - 0s 154us/step - loss: 94133.4930 - val_loss: 2582123.1937
Epoch 7/1000
165/165 [==============================] - 0s 155us/step - loss: 78035.1917 - val_loss: 2356826.3750
Epoch 8/1000
165/165 [==============================] - 0s 152us/step - loss: 62533.9790 - val_loss: 1953561.3600
Epoch 9/1000
165/165 [=================

In [5]:
model_02 = Sequential()

model_02.add(layers.Dense(units=1024, input_shape=(4,)))
model_02.add(Dropout(0.1))
model_02.add(layers.Dense(units=1024))
model_02.add(Dropout(0.1))
model_02.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_02.compile(optimizer='Adagrad', loss='mean_squared_error')
model_02.fit(train_input_02, train_target_02, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 164 samples, validate on 71 samples
Epoch 1/1000
164/164 [==============================] - 0s 1ms/step - loss: 1069296.1677 - val_loss: 3207506.6461
Epoch 2/1000
164/164 [==============================] - 0s 142us/step - loss: 443524.1804 - val_loss: 4173350.8820
Epoch 3/1000
164/164 [==============================] - 0s 147us/step - loss: 296348.7718 - val_loss: 3120827.3028
Epoch 4/1000
164/164 [==============================] - 0s 143us/step - loss: 207463.6657 - val_loss: 2487742.7042
Epoch 5/1000
164/164 [==============================] - 0s 145us/step - loss: 140099.9213 - val_loss: 1866330.7887
Epoch 6/1000
164/164 [==============================] - 0s 157us/step - loss: 83338.8086 - val_loss: 1196605.3200
Epoch 7/1000
164/164 [==============================] - 0s 154us/step - loss: 43969.7699 - val_loss: 680706.5018
Epoch 8/1000
164/164 [==============================] - 0s 143us/step - loss: 21626.0641 - val_loss: 432893.3121
Epoch 9/1000
164/164 [===================

In [6]:
model_03 = Sequential()

model_03.add(layers.Dense(units=1024, input_shape=(4,)))
model_03.add(Dropout(0.1))
model_03.add(layers.Dense(units=1024))
model_03.add(Dropout(0.1))

model_03.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_03.compile(optimizer='Adagrad', loss='mean_squared_error')
model_03.fit(train_input_03, train_target_03, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 164 samples, validate on 71 samples
Epoch 1/1000
164/164 [==============================] - 0s 1ms/step - loss: 1108749.5899 - val_loss: 1966123.5264
Epoch 2/1000
164/164 [==============================] - 0s 138us/step - loss: 618993.5381 - val_loss: 2242784.0335
Epoch 3/1000
164/164 [==============================] - 0s 163us/step - loss: 283759.0787 - val_loss: 1489272.8754
Epoch 4/1000
164/164 [==============================] - 0s 150us/step - loss: 100286.7465 - val_loss: 843419.1725
Epoch 5/1000
164/164 [==============================] - 0s 147us/step - loss: 35257.7059 - val_loss: 545337.8974
Epoch 6/1000
164/164 [==============================] - 0s 155us/step - loss: 20780.1792 - val_loss: 380786.5414
Epoch 7/1000
164/164 [==============================] - 0s 151us/step - loss: 15217.5377 - val_loss: 316098.4963
Epoch 8/1000
164/164 [==============================] - 0s 139us/step - loss: 13803.5179 - val_loss: 265285.9195
Epoch 9/1000
164/164 [=======================

In [7]:
model_04 = Sequential()

model_04.add(layers.Dense(units=1024, input_shape=(4,)))
model_04.add(Dropout(0.1))
model_04.add(layers.Dense(units=1024))
model_04.add(Dropout(0.1))

model_04.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_04.compile(optimizer='Adagrad', loss='mean_squared_error')
model_04.fit(train_input_04, train_target_04, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 164 samples, validate on 71 samples
Epoch 1/1000
164/164 [==============================] - 0s 1ms/step - loss: 1192562.3659 - val_loss: 1336623.1461
Epoch 2/1000
164/164 [==============================] - 0s 140us/step - loss: 702758.5930 - val_loss: 1132548.1092
Epoch 3/1000
164/164 [==============================] - 0s 164us/step - loss: 168687.0831 - val_loss: 954674.2359
Epoch 4/1000
164/164 [==============================] - 0s 146us/step - loss: 85843.8020 - val_loss: 724935.1514
Epoch 5/1000
164/164 [==============================] - 0s 149us/step - loss: 56009.1012 - val_loss: 531542.3732
Epoch 6/1000
164/164 [==============================] - 0s 149us/step - loss: 50573.5335 - val_loss: 489585.0863
Epoch 7/1000
164/164 [==============================] - 0s 157us/step - loss: 42545.3323 - val_loss: 432342.3609
Epoch 8/1000
164/164 [==============================] - 0s 147us/step - loss: 34211.8974 - val_loss: 417999.7975
Epoch 9/1000
164/164 [=========================

In [8]:
model_05 = Sequential()

model_05.add(layers.Dense(units=1024, input_shape=(4,)))
model_05.add(Dropout(0.1))
model_05.add(layers.Dense(units=1024))
model_05.add(Dropout(0.1))

model_05.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_05.compile(optimizer='Adagrad', loss='mean_squared_error')
model_05.fit(train_input_05, train_target_05, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 165 samples, validate on 71 samples
Epoch 1/1000
165/165 [==============================] - 0s 1ms/step - loss: 1225770.6189 - val_loss: 1424388.3187
Epoch 2/1000
165/165 [==============================] - 0s 149us/step - loss: 765673.6600 - val_loss: 1627494.2782
Epoch 3/1000
165/165 [==============================] - 0s 168us/step - loss: 247132.5150 - val_loss: 1335459.7870
Epoch 4/1000
165/165 [==============================] - 0s 161us/step - loss: 126079.1021 - val_loss: 974231.5493
Epoch 5/1000
165/165 [==============================] - 0s 159us/step - loss: 96782.8669 - val_loss: 746639.4313
Epoch 6/1000
165/165 [==============================] - 0s 162us/step - loss: 68865.4960 - val_loss: 575808.6232
Epoch 7/1000
165/165 [==============================] - 0s 146us/step - loss: 54445.9124 - val_loss: 438261.6241
Epoch 8/1000
165/165 [==============================] - 0s 164us/step - loss: 42678.0626 - val_loss: 360309.2526
Epoch 9/1000
165/165 [=======================

In [9]:
model_06 = Sequential()

model_06.add(layers.Dense(units=1024, input_shape=(4,)))
model_06.add(Dropout(0.1))
model_06.add(layers.Dense(units=1024))
model_06.add(Dropout(0.1))

model_06.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_06.compile(optimizer='Adagrad', loss='mean_squared_error')
model_06.fit(train_input_06, train_target_06, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 165 samples, validate on 71 samples
Epoch 1/1000
165/165 [==============================] - 0s 1ms/step - loss: 1178243.8413 - val_loss: 1788938.7711
Epoch 2/1000
165/165 [==============================] - 0s 142us/step - loss: 488906.2761 - val_loss: 2804858.6831
Epoch 3/1000
165/165 [==============================] - 0s 156us/step - loss: 115600.4384 - val_loss: 2666542.8873
Epoch 4/1000
165/165 [==============================] - 0s 147us/step - loss: 81123.6524 - val_loss: 2511846.4859
Epoch 5/1000
165/165 [==============================] - 0s 147us/step - loss: 70135.7967 - val_loss: 2121865.0211
Epoch 6/1000
165/165 [==============================] - 0s 154us/step - loss: 64213.5630 - val_loss: 2024741.3028
Epoch 7/1000
165/165 [==============================] - 0s 153us/step - loss: 59424.3307 - val_loss: 1856068.2148
Epoch 8/1000
165/165 [==============================] - 0s 149us/step - loss: 54332.9011 - val_loss: 1587520.0915
Epoch 9/1000
165/165 [===================

In [10]:
model_07 = Sequential()

model_07.add(layers.Dense(units=1024, input_shape=(4,)))
model_07.add(Dropout(0.1))
model_07.add(layers.Dense(units=1024))
model_07.add(Dropout(0.1))
                          

model_07.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_07.compile(optimizer='Adagrad', loss='mean_squared_error')
model_07.fit(train_input_07, train_target_07, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 165 samples, validate on 72 samples
Epoch 1/1000
165/165 [==============================] - 0s 1ms/step - loss: 1130930.4417 - val_loss: 2803159.6111
Epoch 2/1000
165/165 [==============================] - 0s 131us/step - loss: 415611.0414 - val_loss: 3811587.8889
Epoch 3/1000
165/165 [==============================] - 0s 141us/step - loss: 104387.6343 - val_loss: 3997307.5000
Epoch 4/1000
165/165 [==============================] - 0s 149us/step - loss: 69487.7028 - val_loss: 3981185.8889
Epoch 5/1000
165/165 [==============================] - 0s 154us/step - loss: 68443.0022 - val_loss: 3295541.1667
Epoch 6/1000
165/165 [==============================] - 0s 159us/step - loss: 55422.7132 - val_loss: 2880344.4722
Epoch 7/1000
165/165 [==============================] - 0s 147us/step - loss: 50604.6571 - val_loss: 2863342.2222
Epoch 8/1000
165/165 [==============================] - 0s 147us/step - loss: 47370.8098 - val_loss: 2579937.7778
Epoch 9/1000
165/165 [===================

In [11]:
model_08 = Sequential()

model_08.add(layers.Dense(units=1024, input_shape=(4,)))
model_08.add(Dropout(0.1))
model_08.add(layers.Dense(units=1024))
model_08.add(Dropout(0.1))

model_08.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_08.compile(optimizer='Adagrad', loss='mean_squared_error')
model_08.fit(train_input_08, train_target_08, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 165 samples, validate on 71 samples
Epoch 1/1000
165/165 [==============================] - 0s 1ms/step - loss: 1125994.3598 - val_loss: 3198748.3662
Epoch 2/1000
165/165 [==============================] - 0s 141us/step - loss: 329968.3777 - val_loss: 5036800.2113
Epoch 3/1000
165/165 [==============================] - 0s 162us/step - loss: 117046.6744 - val_loss: 5482092.7887
Epoch 4/1000
165/165 [==============================] - 0s 161us/step - loss: 84290.7683 - val_loss: 4688402.8592
Epoch 5/1000
165/165 [==============================] - 0s 151us/step - loss: 71031.1839 - val_loss: 4223907.8873
Epoch 6/1000
165/165 [==============================] - 0s 158us/step - loss: 62073.0542 - val_loss: 4096097.2817
Epoch 7/1000
165/165 [==============================] - 0s 154us/step - loss: 55524.1188 - val_loss: 3540640.9859
Epoch 8/1000
165/165 [==============================] - 0s 159us/step - loss: 48490.8853 - val_loss: 3165495.2324
Epoch 9/1000
165/165 [===================

In [12]:
model_09 = Sequential()

model_09.add(layers.Dense(units=1024, input_shape=(4,)))
model_09.add(Dropout(0.1))
model_09.add(layers.Dense(units=1024))
model_09.add(Dropout(0.1))

model_09.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_09.compile(optimizer='Adagrad', loss='mean_squared_error')
model_09.fit(train_input_09, train_target_09, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 166 samples, validate on 72 samples
Epoch 1/1000
166/166 [==============================] - 0s 1ms/step - loss: 1097882.9262 - val_loss: 3696219.3056
Epoch 2/1000
166/166 [==============================] - 0s 138us/step - loss: 314955.9674 - val_loss: 4681134.1667
Epoch 3/1000
166/166 [==============================] - 0s 150us/step - loss: 208661.1493 - val_loss: 4468978.5556
Epoch 4/1000
166/166 [==============================] - 0s 140us/step - loss: 157084.9226 - val_loss: 4274307.3333
Epoch 5/1000
166/166 [==============================] - 0s 148us/step - loss: 117679.2956 - val_loss: 3397781.9444
Epoch 6/1000
166/166 [==============================] - 0s 154us/step - loss: 87955.8797 - val_loss: 2738543.8889
Epoch 7/1000
166/166 [==============================] - 0s 158us/step - loss: 64803.1157 - val_loss: 2229158.6389
Epoch 8/1000
166/166 [==============================] - 0s 152us/step - loss: 50732.2683 - val_loss: 1795279.7222
Epoch 9/1000
166/166 [=================

In [13]:
model_10 = Sequential()

model_10.add(layers.Dense(units=1024, input_shape=(4,)))
model_10.add(Dropout(0.1))
model_10.add(layers.Dense(units=1024))
model_10.add(Dropout(0.1))

model_10.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_10.compile(optimizer='Adagrad', loss='mean_squared_error')
model_10.fit(train_input_10, train_target_10, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 166 samples, validate on 72 samples
Epoch 1/1000
166/166 [==============================] - 0s 1ms/step - loss: 1116647.0945 - val_loss: 3655267.8056
Epoch 2/1000
166/166 [==============================] - 0s 132us/step - loss: 433518.8202 - val_loss: 4672765.3889
Epoch 3/1000
166/166 [==============================] - 0s 156us/step - loss: 293202.9415 - val_loss: 3614564.9444
Epoch 4/1000
166/166 [==============================] - 0s 145us/step - loss: 196808.0195 - val_loss: 2643912.4167
Epoch 5/1000
166/166 [==============================] - 0s 163us/step - loss: 124102.8092 - val_loss: 1793210.0556
Epoch 6/1000
166/166 [==============================] - 0s 156us/step - loss: 67866.5727 - val_loss: 1119008.7639
Epoch 7/1000
166/166 [==============================] - 0s 155us/step - loss: 41607.1587 - val_loss: 698225.0556
Epoch 8/1000
166/166 [==============================] - 0s 153us/step - loss: 24136.9485 - val_loss: 430933.3021
Epoch 9/1000
166/166 [===================

In [14]:
model_11 = Sequential()

model_11.add(layers.Dense(units=1024, input_shape=(4,)))
model_11.add(Dropout(0.1))
model_11.add(layers.Dense(units=1024))
model_11.add(Dropout(0.1))

model_11.add(Dense(units=1, kernel_regularizer=tf.keras.regularizers.l1(0.01)))

model_11.compile(optimizer='Adagrad', loss='mean_squared_error')
model_11.fit(train_input_11, train_target_11, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

Train on 165 samples, validate on 72 samples
Epoch 1/1000
165/165 [==============================] - 0s 1ms/step - loss: 1166259.5528 - val_loss: 2976127.8056
Epoch 2/1000
165/165 [==============================] - 0s 147us/step - loss: 606881.2803 - val_loss: 2590182.1667
Epoch 3/1000
165/165 [==============================] - 0s 157us/step - loss: 357603.7542 - val_loss: 1428083.6389
Epoch 4/1000
165/165 [==============================] - 0s 163us/step - loss: 136691.7779 - val_loss: 713690.4514
Epoch 5/1000
165/165 [==============================] - 0s 140us/step - loss: 41656.3385 - val_loss: 282522.1111
Epoch 6/1000
165/165 [==============================] - 0s 155us/step - loss: 13277.5911 - val_loss: 138169.2049
Epoch 7/1000
165/165 [==============================] - 0s 162us/step - loss: 8845.7679 - val_loss: 116867.6441
Epoch 8/1000
165/165 [==============================] - 0s 157us/step - loss: 7395.9075 - val_loss: 83480.3655
Epoch 9/1000
165/165 [==========================

In [15]:
# model_12 = Sequential()

# model_12.add(layers.Dense(units=1024, input_shape=(4,)))
# model_12.add(Dropout(0.1))

# model_12.add(layers.Dense(units=128, input_shape=(4,)))
# model_12.add(Dropout(0.1))

# model_12.add(Dense(units=1))

# model_12.compile(optimizer='Adagrad', loss='mean_squared_error')

-----------------------

In [16]:
RMSE=[]

predicted_01 = model_01.predict(test_input_01)
actual_01=data_test_01['y']
RMSE_01 = mean_squared_error(actual_01, predicted_01)**0.5    
RMSE.append(RMSE_01)
predicted_02 = model_02.predict(test_input_02)
actual_02=data_test_02['y']
RMSE_02 = mean_squared_error(actual_02, predicted_02)**0.5    
RMSE.append(RMSE_02)
predicted_03 = model_03.predict(test_input_03)
actual_03=data_test_03['y']
RMSE_03 = mean_squared_error(actual_03, predicted_03)**0.5    
RMSE.append(RMSE_03)
predicted_04 = model_04.predict(test_input_04)
actual_04=data_test_04['y']
RMSE_04 = mean_squared_error(actual_04, predicted_04)**0.5    
RMSE.append(RMSE_04)
predicted_05 = model_05.predict(test_input_05)
actual_05=data_test_05['y']
RMSE_05 = mean_squared_error(actual_05, predicted_05)**0.5    
RMSE.append(RMSE_05)
predicted_06 = model_06.predict(test_input_06)
actual_06=data_test_06['y']
RMSE_06 = mean_squared_error(actual_06, predicted_06)**0.5    
RMSE.append(RMSE_06)
predicted_07= model_07.predict(test_input_07)
actual_07=data_test_07['y']
RMSE_07 = mean_squared_error(actual_07, predicted_07)**0.5    
RMSE.append(RMSE_07)
predicted_08 = model_08.predict(test_input_08)
actual_08=data_test_08['y']
RMSE_08 = mean_squared_error(actual_08, predicted_08)**0.5    
RMSE.append(RMSE_08)
predicted_09 = model_09.predict(test_input_09)
actual_09=data_test_09['y']
RMSE_09 = mean_squared_error(actual_09, predicted_09)**0.5    
RMSE.append(RMSE_09)
predicted_10 = model_10.predict(test_input_10)
actual_10=data_test_10['y']
RMSE_10 = mean_squared_error(actual_10, predicted_10)**0.5    
RMSE.append(RMSE_10)
predicted_11 = model_11.predict(test_input_11)
actual_11=data_test_11['y']
RMSE_11 = mean_squared_error(actual_11, predicted_11)**0.5    
RMSE.append(RMSE_11)
#predicted_12 = model_12.predict(test_input_12)
#actual_12=data_test_12['y']

#RMSE_12 = mean_squared_error(actual_12, predicted_12)**0.5    
#RMSE_12

predicted_01 = sum(predicted_01.tolist(), [])
actual_01=actual_01.tolist()
final_01=pd.DataFrame([predicted_01, actual_01]).T
final_01.columns=['pred','actual']
final_01=final_01.set_index(data_test_01.index)

predicted_02 = sum(predicted_02.tolist(), [])
actual_02=actual_02.tolist()
final_02=pd.DataFrame([predicted_02, actual_02]).T
final_02.columns=['pred','actual']
final_02=final_02.set_index(data_test_02.index)

predicted_03 = sum(predicted_03.tolist(), [])
actual_03=actual_03.tolist()
final_03=pd.DataFrame([predicted_03, actual_03]).T
final_03.columns=['pred','actual']
final_03=final_03.set_index(data_test_03.index)

predicted_04 = sum(predicted_04.tolist(), [])
actual_04=actual_04.tolist()
final_04=pd.DataFrame([predicted_04, actual_04]).T
final_04.columns=['pred','actual']
final_04=final_04.set_index(data_test_04.index)

predicted_05 = sum(predicted_05.tolist(), [])
actual_05=actual_05.tolist()
final_05=pd.DataFrame([predicted_05, actual_05]).T
final_05.columns=['pred','actual']
final_05=final_05.set_index(data_test_05.index)

predicted_06 = sum(predicted_06.tolist(), [])
actual_06=actual_06.tolist()
final_06=pd.DataFrame([predicted_06, actual_06]).T
final_06.columns=['pred','actual']
final_06=final_06.set_index(data_test_06.index)

predicted_07 = sum(predicted_07.tolist(), [])
actual_07=actual_07.tolist()
final_07=pd.DataFrame([predicted_07, actual_07]).T
final_07.columns=['pred','actual']
final_07=final_07.set_index(data_test_07.index)

predicted_08 = sum(predicted_08.tolist(), [])
actual_08=actual_08.tolist()
final_08=pd.DataFrame([predicted_08, actual_08]).T
final_08.columns=['pred','actual']
final_08=final_08.set_index(data_test_08.index)

predicted_09 = sum(predicted_09.tolist(), [])
actual_09=actual_09.tolist()
final_09=pd.DataFrame([predicted_09, actual_09]).T
final_09.columns=['pred','actual']
final_09=final_09.set_index(data_test_09.index)

predicted_10 = sum(predicted_10.tolist(), [])
actual_10=actual_10.tolist()
final_10=pd.DataFrame([predicted_10, actual_10]).T
final_10.columns=['pred','actual']
final_10=final_10.set_index(data_test_10.index)

predicted_11 = sum(predicted_11.tolist(), [])
actual_11=actual_11.tolist()
final_11=pd.DataFrame([predicted_11, actual_11]).T
final_11.columns=['pred','actual']
final_11=final_11.set_index(data_test_11.index)

#redicted_12 = sum(predicted_12.tolist(), [])
#ctual_12=actual_12.tolist()
#inal_12=pd.DataFrame([predicted_12, actual_12]).T
#inal_12.columns=['pred','actual']
#inal_12=final_12.set_index(data_test_12.index)

#train set의 마지막 데이터
lastday_01=rate_train_01[-1:]['rate']
lastday_02=rate_train_02[-1:]['rate']
lastday_03=rate_train_03[-1:]['rate']
lastday_04=rate_train_04[-1:]['rate']
lastday_05=rate_train_05[-1:]['rate']
lastday_06=rate_train_06[-1:]['rate']
lastday_07=rate_train_07[-1:]['rate']
lastday_08=rate_train_08[-1:]['rate']
lastday_09=rate_train_09[-1:]['rate']
lastday_10=rate_train_10[-1:]['rate']
lastday_11=rate_train_11[-1:]['rate']
#lastday_12=rate_train_12[-1:]['rate']
actual_updown_01=[]
actual_updown_02=[]
actual_updown_03=[]
actual_updown_04=[]
actual_updown_05=[]
actual_updown_06=[]
actual_updown_07=[]
actual_updown_08=[]
actual_updown_09=[]
actual_updown_10=[]
actual_updown_11=[]
actual_updown_12=[]
pred_updown_01=[]
pred_updown_02=[]
pred_updown_03=[]
pred_updown_04=[]
pred_updown_05=[]
pred_updown_06=[]
pred_updown_07=[]
pred_updown_08=[]
pred_updown_09=[]
pred_updown_10=[]
pred_updown_11=[]
pred_updown_12=[]
if final_01['actual'][0] > lastday_01[0]:
    actual_updown_01.append('up')
else :
    actual_updown_01.append('down')
    
if final_01['pred'][0] > lastday_01[0]:
    pred_updown_01.append('up')
else :
    pred_updown_01.append('down')


for i in range(len(final_01)-1):
    
    if final_01['actual'][i+1] > final_01['actual'][i]:
        actual_updown_01.append('up')
    else :
        actual_updown_01.append('down')
        
for i in range(len(final_01)-1):
    
    if final_01['pred'][i+1] > final_01['actual'][i]:
        pred_updown_01.append('up')
    else :
        pred_updown_01.append('down')
        

        ##
        
        
if final_02['actual'][0] > lastday_02[0]:
    actual_updown_02.append('up')
else :
    actual_updown_02.append('down')
    
if final_02['pred'][0] > lastday_02[0]:
    pred_updown_02.append('up')
else :
    pred_updown_02.append('down')


for i in range(len(final_02)-1):
    
    if final_02['actual'][i+1] > final_02['actual'][i]:
        actual_updown_02.append('up')
    else :
        actual_updown_02.append('down')
        
for i in range(len(final_02)-1):
    
    if final_02['pred'][i+1] > final_02['actual'][i]:
        pred_updown_02.append('up')
    else :
        pred_updown_02.append('down')
        

        
        
        
if final_03['actual'][0] > lastday_03[0]:
    actual_updown_03.append('up')
else :
    actual_updown_03.append('down')
    
if final_03['pred'][0] > lastday_03[0]:
    pred_updown_03.append('up')
else :
    pred_updown_03.append('down')


for i in range(len(final_03)-1):
    
    if final_03['actual'][i+1] > final_03['actual'][i]:
        actual_updown_03.append('up')
    else :
        actual_updown_03.append('down')
        
for i in range(len(final_03)-1):
    
    if final_03['pred'][i+1] > final_03['actual'][i]:
        pred_updown_03.append('up')
    else :
        pred_updown_03.append('down')
        
        
                ##


if final_04['actual'][0] > lastday_04[0]:
    actual_updown_04.append('up')
else :
    actual_updown_04.append('down')
    
if final_04['pred'][0] > lastday_04[0]:
    pred_updown_04.append('up')
else :
    pred_updown_04.append('down')


for i in range(len(final_04)-1):
    
    if final_04['actual'][i+1] > final_04['actual'][i]:
        actual_updown_04.append('up')
    else :
        actual_updown_04.append('down')
        
for i in range(len(final_04)-1):
    
    if final_04['pred'][i+1] > final_04['actual'][i]:
        pred_updown_04.append('up')
    else :
        pred_updown_04.append('down')
        
        
                ##


if final_05['actual'][0] > lastday_05[0]:
    actual_updown_05.append('up')
else :
    actual_updown_05.append('down')
    
if final_05['pred'][0] > lastday_05[0]:
    pred_updown_05.append('up')
else :
    pred_updown_05.append('down')


for i in range(len(final_05)-1):
    
    if final_05['actual'][i+1] > final_05['actual'][i]:
        actual_updown_05.append('up')
    else :
        actual_updown_05.append('down')
        
for i in range(len(final_05)-1):
    
    if final_05['pred'][i+1] > final_05['actual'][i]:
        pred_updown_05.append('up')
    else :
        pred_updown_05.append('down')
        
        
                ##


if final_06['actual'][0] > lastday_06[0]:
    actual_updown_06.append('up')
else :
    actual_updown_06.append('down')
    
if final_06['pred'][0] > lastday_06[0]:
    pred_updown_06.append('up')
else :
    pred_updown_06.append('down')


for i in range(len(final_06)-1):
    
    if final_06['actual'][i+1] > final_06['actual'][i]:
        actual_updown_06.append('up')
    else :
        actual_updown_06.append('down')
        
for i in range(len(final_06)-1):
    
    if final_06['pred'][i+1] > final_06['actual'][i]:
        pred_updown_06.append('up')
    else :
        pred_updown_06.append('down')
        

        
                ##

            
if final_07['actual'][0] > lastday_07[0]:
    actual_updown_07.append('up')
else :
    actual_updown_07.append('down')
    
if final_07['pred'][0] > lastday_07[0]:
    pred_updown_07.append('up')
else :
    pred_updown_07.append('down')


for i in range(len(final_07)-1):
    
    if final_07['actual'][i+1] > final_07['actual'][i]:
        actual_updown_07.append('up')
    else :
        actual_updown_07.append('down')
        
for i in range(len(final_07)-1):
    
    if final_07['pred'][i+1] > final_07['actual'][i]:
        pred_updown_07.append('up')
    else :
        pred_updown_07.append('down')
        

                ##

        
        
if final_08['actual'][0] > lastday_08[0]:
    actual_updown_08.append('up')
else :
    actual_updown_08.append('down')
    
if final_08['pred'][0] > lastday_08[0]:
    pred_updown_08.append('up')
else :
    pred_updown_08.append('down')


for i in range(len(final_08)-1):
    
    if final_08['actual'][i+1] > final_08['actual'][i]:
        actual_updown_08.append('up')
    else :
        actual_updown_08.append('down')
        
for i in range(len(final_08)-1):
    
    if final_08['pred'][i+1] > final_08['actual'][i]:
        pred_updown_08.append('up')
    else :
        pred_updown_08.append('down')
        

        
                ##

        
if final_09['actual'][0] > lastday_09[0]:
    actual_updown_09.append('up')
else :
    actual_updown_09.append('down')
    
if final_09['pred'][0] > lastday_09[0]:
    pred_updown_09.append('up')
else :
    pred_updown_09.append('down')


for i in range(len(final_09)-1):
    
    if final_09['actual'][i+1] > final_09['actual'][i]:
        actual_updown_09.append('up')
    else :
        actual_updown_09.append('down')
        
for i in range(len(final_09)-1):
    
    if final_09['pred'][i+1] > final_09['actual'][i]:
        pred_updown_09.append('up')
    else :
        pred_updown_09.append('down')
        
        
        
                ##


if final_10['actual'][0] > lastday_10[0]:
    actual_updown_10.append('up')
else :
    actual_updown_10.append('down')
    
if final_10['pred'][0] > lastday_10[0]:
    pred_updown_10.append('up')
else :
    pred_updown_10.append('down')


for i in range(len(final_10)-1):
    
    if final_10['actual'][i+1] > final_10['actual'][i]:
        actual_updown_10.append('up')
    else :
        actual_updown_10.append('down')
        
for i in range(len(final_10)-1):
    
    if final_10['pred'][i+1] > final_10['actual'][i]:
        pred_updown_10.append('up')
    else :
        pred_updown_10.append('down')
        

        
        
        
                ##

            
            
if final_11['actual'][0] > lastday_11[0]:
    actual_updown_11.append('up')
else :
    actual_updown_11.append('down')
    
if final_11['pred'][0] > lastday_11[0]:
    pred_updown_11.append('up')
else :
    pred_updown_11.append('down')


for i in range(len(final_11)-1):
    
    if final_11['actual'][i+1] > final_11['actual'][i]:
        actual_updown_11.append('up')
    else :
        actual_updown_11.append('down')
        
for i in range(len(final_11)-1):
    
    if final_11['pred'][i+1] > final_11['actual'][i]:
        pred_updown_11.append('up')
    else :
        pred_updown_11.append('down')
# if final_12['actual'][0] > lastday_12[0]:
#     actual_updown_12.append('up')
# else :
#     actual_updown_12.appedn('down')
    
# if final_12['pred'][0] > lastday_12[0]:
#     pred_updown_12.append('up')
# else :
#     pred_updown_12.append('down')


# for i in range(len(final_12)-1):
    
#     if final_12['actual'][i+1] > final_12['actual'][i]:
#         actual_updown_12.append('up')
#     else :
#         actual_updown_12.append('down')
        
# for i in range(len(final_12)-1):
    
#     if final_12['pred'][i+1] > final_12['actual'][i]:
#         pred_updown_12.append('up')
#     else :
#         pred_updown_12.append('down')
# '

final_01['pred_updown']=pred_updown_01
final_01['actual_updown']=actual_updown_01

final_02['pred_updown']=pred_updown_02
final_02['actual_updown']=actual_updown_02

final_03['pred_updown']=pred_updown_03
final_03['actual_updown']=actual_updown_03

final_04['pred_updown']=pred_updown_04
final_04['actual_updown']=actual_updown_04

final_05['pred_updown']=pred_updown_05
final_05['actual_updown']=actual_updown_05

final_06['pred_updown']=pred_updown_06
final_06['actual_updown']=actual_updown_06

final_07['pred_updown']=pred_updown_07
final_07['actual_updown']=actual_updown_07

final_08['pred_updown']=pred_updown_08
final_08['actual_updown']=actual_updown_08

final_09['pred_updown']=pred_updown_09
final_09['actual_updown']=actual_updown_09

final_10['pred_updown']=pred_updown_10
final_10['actual_updown']=actual_updown_10

final_11['pred_updown']=pred_updown_11
final_11['actual_updown']=actual_updown_11

#final_12['pred_updown']=pred_updown_12
#final_12['actual_updown']=actual_updown_12

day5=[]
month=[]
month.append(round(sum(final_01['pred_updown'] == final_01['actual_updown'])/len(final_01)*100,2))
final_5_01=final_01[:5]
day5.append(round(sum(final_5_01['pred_updown'] == final_5_01['actual_updown'])/len(final_5_01)*100,2))
month.append(round(sum(final_02['pred_updown'] == final_02['actual_updown'])/len(final_02)*100,2))
final_5_02=final_02[:5]
day5.append(round(sum(final_5_02['pred_updown'] == final_5_02['actual_updown'])/len(final_5_02)*100,2))
month.append(round(sum(final_03['pred_updown'] == final_03['actual_updown'])/len(final_03)*100,2))
final_5_03=final_03[:5]
day5.append(round(sum(final_5_03['pred_updown'] == final_5_03['actual_updown'])/len(final_5_03)*100,2))
month.append(round(sum(final_04['pred_updown'] == final_04['actual_updown'])/len(final_04)*100,2))
final_5_04=final_04[:5]
day5.append(round(sum(final_5_04['pred_updown'] == final_5_04['actual_updown'])/len(final_5_04)*100,2))
month.append(round(sum(final_05['pred_updown'] == final_05['actual_updown'])/len(final_05)*100,2))
final_5_05=final_05[:5]
day5.append(round(sum(final_5_05['pred_updown'] == final_5_05['actual_updown'])/len(final_5_05)*100,2))
month.append(round(sum(final_06['pred_updown'] == final_06['actual_updown'])/len(final_06)*100,2))
final_5_06=final_06[:5]
day5.append(round(sum(final_5_06['pred_updown'] == final_5_06['actual_updown'])/len(final_5_06)*100,2))
month.append(round(sum(final_07['pred_updown'] == final_07['actual_updown'])/len(final_07)*100,2))
final_5_07=final_07[:5]
day5.append(round(sum(final_5_07['pred_updown'] == final_5_07['actual_updown'])/len(final_5_07)*100,2))
month.append(round(sum(final_08['pred_updown'] == final_08['actual_updown'])/len(final_08)*100,2))
final_5_08=final_08[:5]
day5.append(round(sum(final_5_08['pred_updown'] == final_5_08['actual_updown'])/len(final_5_08)*100,2))
month.append(round(sum(final_09['pred_updown'] == final_09['actual_updown'])/len(final_09)*100,2))
final_5_09=final_09[:5]
day5.append(round(sum(final_5_09['pred_updown'] == final_5_09['actual_updown'])/len(final_5_09)*100,2))
month.append(round(sum(final_10['pred_updown'] == final_10['actual_updown'])/len(final_10)*100,2))
final_5_10=final_10[:5]
day5.append(round(sum(final_5_10['pred_updown'] == final_5_10['actual_updown'])/len(final_5_10)*100,2))
month.append(round(sum(final_11['pred_updown'] == final_11['actual_updown'])/len(final_11)*100,2))
final_5_11=final_11[:5]
day5.append(round(sum(final_5_11['pred_updown'] == final_5_11['actual_updown'])/len(final_5_11)*100,2))

# print('set12')
# # 한달간 예측
# print('한달간 상승,하락 정확도 : ',round(sum(final_12['pred_updown'] == final_12['actual_updown'])/len(final_12)*100,2), '%')
# #5일만 예측
# final_5_12=final_12[:5]
# print('5일간 상승,하락 정확도 : ',round(sum(final_5_12['pred_updown'] == final_5_12['actual_updown'])/len(final_5_12)*100,2), '%')

In [17]:
print('RMSE평균 : ',np.mean(RMSE))
RMSE

RMSE평균 :  31.165023339964353


[4.544493305558763,
 18.869483424519995,
 12.597135713120279,
 37.443381853528024,
 21.611272000935138,
 43.046466893297776,
 53.48147545351258,
 33.55545207135169,
 75.56843884665943,
 34.654405143117266,
 7.443252034006965]

In [18]:
print('5일 업다운 정확도 : ' ,np.mean(day5))
day5

5일 업다운 정확도 :  36.36363636363637


[60.0, 100.0, 40.0, 0.0, 80.0, 20.0, 0.0, 0.0, 20.0, 20.0, 60.0]

In [19]:
print('한달 업다운 정확도: ' ,np.mean(month))
month

한달 업다운 정확도:  44.89272727272727


[46.67, 68.42, 30.0, 15.79, 77.78, 38.1, 35.0, 47.06, 35.0, 52.63, 47.37]